In [1]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import faiss
import os, io, pickle
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    SpacyTextSplitter
)
from langchain.vectorstores import FAISS
import numpy as np
import pandas as pd
import tiktoken
from sentence_transformers import SentenceTransformer
import shutil
import statistics
import json
import os
from dotenv import load_dotenv

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate_google():
    creds = None
    if os.path.exists("token"):
        with open("token", "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials3.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token", "wb") as token:
            pickle.dump(creds, token)
    
    return build("drive", "v3", credentials=creds)

def download_pdf(file_id, output_path):
    service = authenticate_google()
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(output_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Téléchargement : {int(status.progress() * 100)}%")
    print(f"Fichier téléchargé : {output_path}")


file_id = "1AHE1lXi_kyrtRw31qEGJ7OYE9EO8kfGe"
download_pdf(file_id, "Histoire_CM1.pdf")

Téléchargement : 100%
Fichier téléchargé : Histoire_CM1.pdf


In [3]:
# Fonction pour estimer le nombre de tokens d’un texte
def count_tokens(text, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

In [4]:
# Charger ton PDF
loader = PyPDFLoader("Histoire_CM1.pdf",
                     mode = "page", # Extract the PDF by page. Each page is extracted as a langchain Document object
                     # mode = "single" # PyPDFLoader will split the PDF as a single text flow
                     )
docs = loader.load()

In [5]:
full_text = "\n".join([page.page_content for page in docs])
full_text

'LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les grottes ornées \n \n*En 1940, 4 enfants découvrent une grotte recouverte \nde peintures  : des taureaux, des cerfs, des chevaux…  : \nla grotte de Lascaux.  En datant les objets trouvés dans la grotte on \nsait qu’elle a été peinte il y a environ 17000 ans.  \n \n*En 1994, Jean -Marie Chauvet découvre une autre grotte pei

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(full_text)
token_counts = [count_tokens(chunk) for chunk in chunks]

print(f"Nombre de chunks : {len(chunks)}")
print(f"Tokens/chunk (moy) : {round(statistics.mean(token_counts))}")
print(f"Tokens max : {max(token_counts)}")
print(f"Aperçu du 1er chunk :\n{chunks[0]}")


Nombre de chunks : 27
Tokens/chunk (moy) : 144
Tokens max : 161
Aperçu du 1er chunk :
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.


In [7]:
chunks

['LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.',
 'l’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les grottes ornées \n \n*En 1940, 4 enfants découvrent une grotte recouverte \nde peintures  : des taureaux, des cerfs, des chevaux…  : \nla grotte de Lascaux.  En datant les objets trouvés dans la grotte on \nsait qu’elle a été peinte il y a environ 17000 ans.',
 '*En 1994, Jean -Marie Chauvet découvre une a

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

print("Taille des embeddings : ", embeddings.shape)
for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
    print("="*60)
    print(f"Chunk {i+1}:")
    print(chunk)
    print(f"\n Embedding (taille {len(emb)}):")
    print(np.round(emb[:10], 3))  # Affiche les 10 premières valeurs, arrondies pour lisibilité
    print("="*60)

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Taille des embeddings :  (27, 384)
Chunk 1:
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.

 Embedding (taille 384):
[-0.024  0.15   0.059 -0.052 -0.017  0.151 -0.052  0.037 -0.035  0.003]
Chunk 2:
l’Histoire.  
*L’Histoire de France est divisée en 5 périodes : 
l’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème 
siècle – le XXème siècle. 
 
 
 
 
2) Des traces du passé : les grottes ornées 
 
*En 1940, 4 enfants découvrent une grotte recouverte 
de peintures  : des taureaux, des cerfs, des chevaux…  : 
la grotte de Lascaux.  En datant les objets trouvés d

In [9]:
import shutil
import os

chroma_db_path = "chroma_db"

if os.path.exists(chroma_db_path):
    shutil.rmtree(chroma_db_path)

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
documents = [Document(page_content=chunk, metadata={"chunk_id": i}) for i, chunk in enumerate(chunks)]

chroma_db = Chroma.from_documents(
    documents, 
    embedding_model, 
    persist_directory = chroma_db_path)

/tmp/ipykernel_1780/1402809748.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [10]:
vector_db_data = chroma_db.get()
len(vector_db_data['ids'])

27

In [11]:
documents

[Document(metadata={'chunk_id': 0}, page_content='LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.'),
 Document(metadata={'chunk_id': 1}, page_content='l’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les grottes ornées \n \n*En 1940, 4 enfants découvrent une grotte recouverte \nde peintures  : des taureaux, des cerfs, des chevaux…  : \nla grotte de Lascaux.  En datant les objets trouvés dans la grotte on \nsai

In [12]:
chroma_db

In [13]:
# Test de recherche
query = "Qui est Napoléon ?"

# Récupère les chunks les plus pertinents associés à la requête
# Exemple avec une recherche basée sur un score de similarité
retriever = chroma_db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs = {"k": 3, "score_threshold": 0.2},
)

relevant_docs = retriever.invoke(query)
relevant_docs

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(id='6e3b76ee-3a9b-4dbb-ae79-bde367be3ea6', metadata={'chunk_id': 23}, page_content='autre gouvernement est mis en place.  \n \n*Il sera renversé par Napoléon le 18 brumaire.   \n \n  *Constitution : texte qui précise comment un pays est dirigé.\n21) Le Consulat et l’Empire (1799-1815) \n \n*Le général Bonaparte remporte de nombreuses victoires \nmilitaires, comme Austerlitz.  \n \n*Il  adopte une nouvelle constitution qui lui donne tous les \npouvoirs.  \n \n*Il crée les départements dirigés par un préfet, la banque de France, \nles lycées…'),
 Document(id='7f344755-c84d-470d-a045-e5dad2c5c412', metadata={'chunk_id': 25}, page_content='*Napoléon perd la bataille de Waterloo contre les Anglais et abdique \ndéfinitivement en 1815.    \n \n*Abdiquer : renoncer au pouvoir\n22) Les apports de la Révolution et de l’Empire \n \n\uf0b7 La Révolution et l’Empire ont permis de créer une \nadministration efficace.  \n \n\uf0b7 Les Français sont des citoyens avec des droits et des devoir

In [14]:
print("Nombre de chunks pertinents :", len(relevant_docs))
    
# Affiche les résultats pertinents avec les métadonnées associées
print("\n--- Documents les plus pertinents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"chunk_id : {doc.metadata.get('chunk_id')}\n")

Nombre de chunks pertinents : 3

--- Documents les plus pertinents ---
Document 1:
autre gouvernement est mis en place.  
 
*Il sera renversé par Napoléon le 18 brumaire.   
 
  *Constitution : texte qui précise comment un pays est dirigé.
21) Le Consulat et l’Empire (1799-1815) 
 
*Le général Bonaparte remporte de nombreuses victoires 
militaires, comme Austerlitz.  
 
*Il  adopte une nouvelle constitution qui lui donne tous les 
pouvoirs.  
 
*Il crée les départements dirigés par un préfet, la banque de France, 
les lycées…

chunk_id : 23

Document 2:
*Napoléon perd la bataille de Waterloo contre les Anglais et abdique 
définitivement en 1815.    
 
*Abdiquer : renoncer au pouvoir
22) Les apports de la Révolution et de l’Empire 
 
 La Révolution et l’Empire ont permis de créer une 
administration efficace.  
 
 Les Français sont des citoyens avec des droits et des devoirs.  
 
 Les lois sont réunies dans le code civil  et les juges deviennent 
indépendants.  
 
 Chacun est libre 

# MON TEST

In [15]:
context = chroma_db.get()
context

{'ids': ['fc5320e0-eec5-4493-9f94-9b18ca4ed1c2',
  'bf9214e4-0370-49f5-888f-8491f5c2099f',
  'f11b4385-bcb3-4841-a975-758a3d97c9c1',
  '581d7258-6caf-4b98-8a8d-bdd307afc930',
  'f7993f64-1c0b-4b92-890b-d394ef3aee7d',
  '11cc8d0d-6858-43d6-b84a-c216a78b9258',
  '2154f272-6f00-4f05-bcb4-019287fe05f2',
  '39b9db7d-814f-4191-bb32-827973c22941',
  '29b075b0-ce64-4ac6-ac69-9a85b27af5b5',
  'be258c30-e880-4340-b8e7-8d52ec89994a',
  '329032eb-e033-49b8-a8a8-edd2d4fa1d5e',
  '82152699-39ee-4d26-a9c2-fa198dd912c9',
  '5da25612-841b-4aad-8113-12cf966a0dbb',
  'c73874b1-2ecc-4c3e-8f3e-e1371de3ff9d',
  '448bc049-a04c-45f8-8544-e03802187389',
  'fa8c969e-dc77-47f2-94e6-4f625b48db4f',
  'ba6be603-2c1b-41a8-88bc-d4e23bed48cd',
  '542dfa31-34b2-4589-a6f7-7c41f00fd7e3',
  '6331182e-25cb-4b9f-b3c4-77722d4280ef',
  'e4cb6639-d1c6-4897-a291-7afc048dd956',
  'e5abc228-7485-4e68-8407-6dff68a2dcd6',
  'fd052393-be49-4710-af4a-ad3615e7fdaa',
  '8f4eda7e-42d1-411b-b67a-02bb5290990e',
  '6e3b76ee-3a9b-4dbb-ae79-

In [16]:
len(vector_db_data['metadatas'])

27

In [17]:
vector_db_data['documents'][4]

'font souvent la guerre.  \n \n*Ils vivent dans des fermes  regroupées en village. Ce sont des \nagriculteurs, des éleveurs , pour cela ils déboisent pour former des \nchamps qui sont encore présents aujourd’hui.  \n \n*Ce sont également des artisans  habiles qui créent des outils et \ninventent le tonneau, la roue, le savon, la charrue attelée, la faux…\n5) Qui sont les Gallo-Romains ? \n \n*Les Romains, emmenés par Jules César, ont envahi \nla Gaule en l’an 52 av. JC . Les Gaulois deviennent'

In [20]:
from langchain_groq import ChatGroq
# ATTENTION RECREER UNE CLEF https://console.groq.com/keys

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    api_key=groq_api_key,
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_tokens=None,
    max_retries=2,
)

response = llm.invoke("donne moi une question de QCM sur les shtroumpfs")
print(response.content)


Voici une question de QCM sur les Shtroumpfs :

Quel est le nom du Shtroumpf qui est le plus intelligent et qui porte des lunettes ?

A) Shtroumpf Bleu
B) Shtroumpf Vert
C) Grand Shtroumpf
D) Shtroumpf Jaune

Réponse : C) Grand Shtroumpf


# TEST VALIDE et optimal

In [21]:
import time

json_quiz = []
i = 0
total_duration = 0
#niveau_difficulte = user_input.get("niveau_difficulte", "standard")
niveau_difficulte = "standard"

# Inséré dans le prompt
niveau_instruction = {
    "facile": "La question doit être simple, accessible à un élève de collège.",
    "moyen": "La question doit correspondre à un niveau lycée ou début d’université.",
    "difficile": "La question doit être complexe, s’adresser à un étudiant avancé ou expert.",
    "standard": "Utilise un niveau intermédiaire (niveau lycée/université)."
}[niveau_difficulte]

#Ajouter un niveau de difficulté

for doc in vector_db_data['documents']:
    start = time.time()
    i+=1
    prompt_chunk = f"""
    Tu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.

    {niveau_instruction}

    **Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  
    **Tu ne dois en aucun cas utiliser des connaissances extérieures.**

    Contrainte :
    - La question doit porter sur une information claire et importante du texte.
    - Il doit y avoir exactement 4 choix.
    - Les 4 choix doivent être différents.
    - Une seule bonne réponse.
    - Le niveau de difficulté de la question doit correspondre à celui demandé.
    - Tu ne dois pas inventer d'information : toute la question et ses réponses doivent découler directement du texte.
    - **Réponds uniquement avec un objet JSON valide**, sans aucune explication, ni phrase introductive, ni conclusion.

    {{
        "question": "...",
        "choices": {{
            "a": "...",
            "b": "...",
            "c": "...",
            "d": "..."
        }},
        "correct_answer": {{
            "lettre": "...",
            "answer": "..."
        }},
        "difficulty_level": "..."
    }}

    ## Exemples de chuncks et résultats de QCM attendu

    ### Exemple 1
    #### Le chunk
    {{"En 1789, la Révolution française marque un tournant majeur dans l'histoire de France. Elle débute avec la convocation des États généraux par Louis XVI et conduit à l'abolition des privilèges ainsi qu'à la Déclaration des droits de l'homme et du citoyen. Cette période, marquée par une forte instabilité politique, voit l'émergence de nouvelles institutions et l'affirmation du principe de souveraineté nationale au détriment de la monarchie absolue."}}

    #### Le résultat
    {{
        "question": "Quel événement a marqué un tournant majeur dans l'histoire de France en 1789 ?",
        "choices": {{
            "a": "La chute de Napoléon",
            "b": "La Révolution française",
            "c": "La guerre franco-prussienne",
            "d": "Le couronnement de Louis XVI"
        }},
        "correct_answer": {{
            "lettre": "b",
            "answer": "La Révolution française"
        }}, 
        "difficulty_level": "standard"
    }}

    ### Exemple 2
    #### Le chunk
    {{"L’eau change d’état selon la température. À 0 °C, elle peut passer de l’état liquide à l’état solide (glace) ou inversement. À 100 °C, elle passe de l’état liquide à l’état gazeux (vapeur). Ces transitions sont appelées changements d’état physique."}}

    #### Le résultat
    {{
        "question": "Quel est le nom du phénomène où l'eau passe de l'état liquide à l'état gazeux à 100 °C ?",
        "choices": {{
            "a": "La condensation",
            "b": "La solidification",
            "c": "La vaporisation",
            "d": "La fusion"
        }},
        "correct_answer": {{
            "lettre": "c",
            "answer": "La vaporisation"
        }}, 
        "difficulty_level": "difficile"
    }}

    ### Exemple 3
    #### Le chunk
    {{"L’Amazonie est une vaste forêt tropicale qui s'étend sur neuf pays d'Amérique du Sud, principalement au Brésil. Elle joue un rôle crucial dans la régulation du climat mondial et abrite une biodiversité exceptionnelle. Cette région est souvent qualifiée de poumon vert de la planète."}}

    #### Le résultat
    {{
        "question": "Quel rôle joue l'Amazonie dans le climat mondial ?",
        "choices": {{
            "a": "Elle produit des vents froids",
            "b": "Elle régule le climat mondial",
            "c": "Elle empêche les tsunamis",
            "d": "Elle bloque les courants marins"
        }},
        "correct_answer": {{
            "lettre": "b",
            "answer": "Elle régule le climat mondial"
        }}, 
        "difficulty_level": "facile"
    }}
    
    Texte :
    \"\"\"
    {doc}
    \"\"\"
    """

    response = llm.invoke(prompt_chunk)

    try:
        parsed = json.loads(response.content)
        json_quiz.append(parsed)
    except:
        print("Texte retourné :", response.content[:200])
        continue 
    
    end = time.time()
    duration = round(end - start, 2)
    total_duration += duration
    print(i, ':', duration, 's')

print(total_duration)
json_quiz

1 : 1.74 s
2 : 2.22 s
3 : 1.22 s
4 : 0.61 s
5 : 4.93 s
6 : 9.57 s
7 : 16.1 s
8 : 11.92 s
9 : 25.13 s
10 : 3.01 s
11 : 11.65 s
12 : 13.47 s
13 : 14.33 s
14 : 13.89 s
15 : 11.59 s
16 : 14.16 s
17 : 12.59 s
18 : 15.73 s
19 : 12.41 s
20 : 11.8 s
21 : 14.2 s
22 : 13.82 s
23 : 14.26 s
24 : 13.02 s
25 : 14.13 s
26 : 12.46 s
27 : 13.1 s
303.06


[{'question': "Qu'est-ce que l'Histoire ?",
  'choices': {'a': "L'étude de notre présent pour comprendre notre avenir",
   'b': "L'étude de notre passé pour mieux comprendre notre vie aujourd'hui",
   'c': 'La représentation du temps par une ligne graduée',
   'd': "La période avant l'invention de l'écriture"},
  'correct_answer': {'lettre': 'b',
   'answer': "L'étude de notre passé pour mieux comprendre notre vie aujourd'hui"},
  'difficulty_level': 'standard'},
 {'question': "Quel est le nombre de périodes dans l'Histoire de France ?",
  'choices': {'a': '3', 'b': '4', 'c': '5', 'd': '6'},
  'correct_answer': {'lettre': 'c', 'answer': '5'},
  'difficulty_level': 'standard'},
 {'question': "Quel est l'âge approximatif de la grotte Chauvet lors de sa découverte par Jean-Marie Chauvet ?",
  'choices': {'a': '20 000 ans',
   'b': '30 000 ans',
   'c': '40 000 ans',
   'd': '36 000 ans'},
  'correct_answer': {'lettre': 'd', 'answer': '36 000 ans'},
  'difficulty_level': 'standard'},
 {'qu